Following the post by [Michael Wy Ong](https://medium.com/@michael.wy.ong/web-scrape-geospatial-data-analyse-singapores-property-price-part-i-276caba320b)

In [3]:
import pandas as pd
import numpy as np
import requests
import json

In [32]:
## Function for getting postal code, geo coordinates of addresses

def find_postal(lst, filename):
    '''With the block number and street name, get the full address of the hdb flat,
    including the postal code, geogaphical coordinates (lat/long)'''
    
    for index,add in enumerate(lst):
        # Do not need to change the URL
        url= "https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal="+ add        
        print(index,url)
        
        # Retrieve information from website
        response = requests.get(url)
        try:
            data = json.loads(response.text) 
        except ValueError:
            print('JSONDecodeError')
            pass
    
        temp_df = pd.DataFrame.from_dict(data["results"])
        # The "add" is the address that was used to search in the website
        temp_df["address"] = add
        
        # Create the file with the first row that is read in 
        if index == 0:
            file = temp_df
        else:
            file = file.append(temp_df)
    file.to_csv(filename + '.csv')

In [4]:
price1999 = pd.read_csv('Data/resale-flat-prices-based-on-approval-date-1990-1999.csv')
price2012 = pd.read_csv('Data/resale-flat-prices-based-on-approval-date-2000-feb-2012.csv')
price2014 = pd.read_csv('Data/resale-flat-prices-based-on-registration-date-from-mar-2012-to-dec-2014.csv')
price2016 = pd.read_csv('Data/resale-flat-prices-based-on-registration-date-from-jan-2015-to-dec-2016.csv')
price2017 = pd.read_csv('Data/resale-flat-prices-based-on-registration-date-from-jan-2017-onwards.csv')

prices = pd.concat([price1999, price2012, price2014], sort=False)
prices = pd.concat([prices, price2016, price2017], axis=0, ignore_index=True, sort=False)

In [8]:
prices = prices[['block', 'street_name']]
prices['address'] = prices['block'] + ' ' + prices['street_name']
all_address = list(prices['address'])
unique_address = list(set(all_address))

print('Unique addresses:', len(unique_address))

Unique addresses: 9279


In [10]:
unique_address[:10]

['754 WOODLANDS CIRCLE',
 '481 SEMBAWANG DR',
 '506 BEDOK NTH AVE 3',
 '688B CHOA CHU KANG DR',
 '674 CHOA CHU KANG CRES',
 '306 SERANGOON AVE 2',
 '673 HOUGANG AVE 8',
 '247 ANG MO KIO AVE 3',
 '525C PASIR RIS ST 51',
 '120 PENDING RD']

In [13]:
find_postal(unique_address, 'Data/flat_coordinates')

s.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=108 BEDOK NTH RD
9113 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=540 HOUGANG AVE 8
9114 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=429 CHOA CHU KANG AVE 4
9115 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=2 HAIG RD
9116 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=344 CHOA CHU KANG LOOP
9117 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=29 CHAI CHEE AVE
9118 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=592A MONTREAL LINK
9119 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=294 TAMPINES ST 22
9120 https://developers.onemap.sg/commonapi/search?returnGeom=Y&get

In [15]:
supermarket = pd.read_csv('Data/list-of-supermarket-licences.csv')
supermarket.head()

,licence_num,licensee_name,building_name,block_house_num,level_num,unit_num,street_name,postal_code
0,S02185J000,COLD STORAGE SINGAPORE (1983) PTE LTD,na,982,1,1,BUANGKOK CRESCENT,530982
1,S97166P000,COLD STORAGE SINGAPORE (1983) PTE LTD,HOUGANG GREEN SHOPPING MALL,21,1,52,HOUGANG STREET 51,538719
2,CE15B61K000,U STARS SUPERMARKET PTE. LTD.,na,330,1,1,ANCHORVALE STREET,540330
3,S02029J000,NTUC Fairprice Co-operative Ltd,HOUGANG POINT,1,2,1,HOUGANG STREET 91,538692
4,CE04561V000,SHENG SIONG SUPERMARKET PTE LTD,na,19,1,42401,SERANGOON NORTH AVENUE 5,554913


In [16]:
supermerket_address = list(supermarket['postal_code'])
unique_supermarket_address = list(set(supermerket_address))

print('Unique addresses:', len(unique_supermarket_address))

Unique addresses: 445


In [17]:
find_postal(unique_supermarket_address, 'Data/supermarket_coordinates')

://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=757713
259 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=238801
260 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=768929
261 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=597071
262 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=608521
263 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=320074
264 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=380114
265 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=460018
266 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=437112
267 https://developers.onemap.sg/commonapi/sea

In [21]:
school = pd.read_csv('Data/school_names.csv', encoding='cp1252')
school.head(10)

,school,area
0,Admiralty Primary School,Woodlands
1,Ahmad Ibrahim Primary School,Yishun
2,Ai Tong School,Bishan
3,Alexandra Primary School,Bukit Merah
4,Anchor Green Primary School,Sengkang
5,Anderson Primary School,Ang Mo Kio
6,Anglo-Chinese School (Junior),Central
7,Anglo-Chinese School (Primary),Bukit Timah
8,Angsana Primary School,Tampines
9,Ang Mo Kio Primary School,Ang Mo Kio


In [22]:
school_name = list(school['school'])
unique_school_name = list(set(school_name))

print('Unique addresses:', len(unique_school_name))

Unique addresses: 191


In [33]:
find_postal(unique_school_name, 'Data/school_coordinates')

berra Primary School
31 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=Riverside Primary School
32 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=Meridian Primary School
33 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=Rosyth School
34 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=Cantonment Primary School
35 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=Westwood Primary School
36 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=Zhangde Primary School
37 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=Fuhua Primary School
38 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=Concord Primary School
39 http

In [35]:
hawker = pd.read_csv('Data/list-of-government-markets-hawker-centres.csv')
hawker.head(10)

,name_of_centre,location_of_centre,type_of_centre,owner,no_of_stalls,no_of_cooked_food_stalls,no_of_mkt_produce_stalls
0,Adam Road Food Centre,"2, Adam Road, S(289876)",HC,Government,32,32,0
1,Amoy Street Food Centre,"National Development Building, Annex B, Telok ...",HC,Government,135,134,1
2,Bedok Food Centre,"1, Bedok Road, S(469572)",HC,Government,32,32,0
3,Beo Crescent Market,"38A, Beo Crescent, S(169982)",MHC,Government,94,32,62
4,Berseh Food Centre,"166, Jalan Besar, S(208877)",HC,Government,66,66,0
5,Bukit Timah Market,"51, Upper Bukit Timah Road, S(588215)",MHC,Government,179,84,95
6,Chomp Chomp Food Centre,"20, Kensington Park Road, S(557269)",HC,Government,36,36,0
7,Commonwealth Crescent Market,"31, Commonwealth Crescent, S(149644)",MHC,Government,102,39,63
8,Dunman Food Centre,"271, Onan Road, S(424768)",HC,Government,30,30,0
9,East Coast Lagoon Food Village,"1220, East Coast Parkway, S(468960)",HC,Government,63,63,0


In [37]:
hawker_name = list(hawker['name_of_centre'])
unique_hawker_name = list(set(hawker_name))

print('Unique addresses:', len(unique_hawker_name))

Unique addresses: 107


In [38]:
find_postal(unique_hawker_name, 'Data/hawker_coordinates')

0 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=Blk 82 Telok Blangah Drive
1 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=Blk 503 West Coast Drive
2 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=Amoy Street Food Centre
3 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=Blks 20/21 Marsiling Lane
4 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=Blk 36 Telok Blangah Rise
5 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=Blks 91/92 Whampoa Drive
6 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=Geylang Serai Market
7 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=Blks 2 & 3 Changi Village Road
8 https://d